In [67]:
using NPZ
using PyPlot
using HDF5
using JLD
using LaTeXStrings
using QuanticsTCI
import TensorCrossInterpolation as TCI

rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
#rcParams["backend"] = :tk
rcParams["font.size"] = 9
rcParams["text.usetex"] = true
rcParams["font.family"] = "serif"
#rcParams["text.latex.preamble"] = [raw"\usepackage{amsmath}", raw"\usepackage{amssymb}"]
rcParams["font.serif"] = ["Computer Modern Roman"]
rcParams["lines.linewidth"] = 1.0
rcParams["lines.markersize"] = 2
rcParams["image.cmap"] = "coolwarm"

function maxabs(data)
    return maximum(abs.(data))
end


maxabs (generic function with 1 method)

In [6]:
u_here = 0.5
u_here = 1.
keldyshComponentsK2a = "1112"
keldyshComponentsK3a = "1212"
idx_keldyshComponentsK2a = 2
idx_keldyshComponentsK3a = 4
#=
Keldyshcomponents       in K2a:     in K3a:
                        1111        1111
                        1112        1112
                        1121        1122
                        1122        1212
                        2122        1221
                                    1222
=#

filename = "/home/Anxiang.Ge/Desktop/PhD/mfrg_Data/Keldysh/1lpaper_data/parquetInit4_U_over_Delta=1.570796_T=0.010000_eVg=0.000000_n1=401_n2=201_n3=51_version1_final.h5"
filename = "/home/Anxiang.Ge/Desktop/PhD/mfrg_Data/Keldysh/1lpaper_data/parquetInit4_U_over_Delta=3.141593_T=0.010000_eVg=0.000000_n1=401_n2=201_n3=101_version1_final.h5"
fparquet = h5open(filename);

parquetK3t = map(t -> t.re + 1im * t.im, fparquet["K3_t"][1, 1, :, :, :, 1, 1])
parquetK3a = map(t -> t.re + 1im * t.im, fparquet["K3_a"][1, idx_keldyshComponentsK3a, :, :, :, 1, 1])
parquetK3p = map(t -> t.re + 1im * t.im, fparquet["K3_p"][1, 1, :, :, :, 1, 1])

parquetK2t = map(t -> t.re + 1im * t.im, fparquet["K2_t"][1, 1, :, :, 1, 1])
parquetK2a = map(t -> t.re + 1im * t.im, fparquet["K2_a"][1, idx_keldyshComponentsK2a, :, :, 1, 1])
parquetK2p = map(t -> t.re + 1im * t.im, fparquet["K2_p"][1, 1, :, :, 1, 1])

parquetK1t = map(t -> t.re + 1im * t.im, fparquet["K1_t"][1, 1, :, 1, 1])
parquetK1a = map(t -> t.re + 1im * t.im, fparquet["K1_a"][1, 1, :, 1, 1])
parquetK1p = map(t -> t.re + 1im * t.im, fparquet["K1_p"][1, 1, :, 1, 1]);

In [7]:
using Interpolations

omegas_K2 = fparquet["bfreqs2_a"][:,1]
nus_K2 = fparquet["ffreqs2_a"][:,1]
omegas_K3 = fparquet["bfreqs3_a"][:,1]
nus_K3 = fparquet["ffreqs3_a"][:,1]

δω_K2 = omegas_K2[div(length(omegas_K2),2)+2]
δν_K2 = nus_K2[div(length(nus_K2),2)+2]
δω_K3 = omegas_K3[div(length(omegas_K3),2)+2]
δν_K3 = nus_K3[div(length(nus_K3),2)+2]

K2ainterp = interpolate((nus_K2, omegas_K2), parquetK2a, Gridded(Linear()))
K3ainterp = interpolate((nus_K3, nus_K3, omegas_K3), parquetK3a, Gridded(Linear()));

In [8]:
extent = 10
plotomegas = range(-extent, extent, length=1000)
plotnus = range(-extent, extent, length=1000)
interpdata_K2 = K2ainterp.(plotomegas, plotnus')
interpdata_K3 = K3ainterp.(plotnus, plotnus', 0.)

vmax = maximum(abs.(parquetK2a))
kwargs = Dict(:vmax=>vmax, :vmin=>-vmax)

fig, axs = subplots(ncols=2, nrows=2)
axs[1, 1].imshow(real(interpdata_K2); kwargs...) |> colorbar
axs[1, 2].imshow(imag(interpdata_K2); kwargs...) |> colorbar
axs[2, 1].imshow(real(interpdata_K3); kwargs...) |> colorbar
axs[2, 2].imshow(imag(interpdata_K3); kwargs...) |> colorbar

plt.savefig("interpdata_K2aK3a.png")

Dict{String, Int64} with 2 entries:
  "R"      => 14
  "extent" => 10

In [85]:
### QTCI K2a: ###


R_K2 = 14
qR_K2 = 2^R_K2
qmesh = range(-extent, extent, length=qR_K2) |> collect
println("extent of qmesh: \t ±", extent)
println("resol of qmesh: \t", (2. *extent) / qR_K2)
println()
println("resol of ω/ν: \t", minimum([δω_K2, δν_K2]))

tol_high = 1e-2
tol_low  = 1e-3


qttK2a, _, _ = quanticscrossinterpolate(
    ComplexF64,
    (x, y) -> K2ainterp(x, y),
    [qmesh, qmesh],
    tolerance=tol_high
)

qttK2a2, _, _ = quanticscrossinterpolate(
    ComplexF64,
    (x, y) -> K2ainterp(x, y),
    [qmesh, qmesh],
    tolerance=tol_low
)


extent of qmesh: 	 ±10
resol of qmesh: 	0.001220703125

resol of ω/ν: 	0.00047744096259767907


(QuanticsTCI.QuanticsTensorCI2{ComplexF64}(TensorCrossInterpolation.TensorCI2{ComplexF64} with rank 102, QuanticsTCI.UnfoldingSchemes.interleaved), [2, 4, 8, 16, 32, 60, 91, 110, 105, 102], [0.0, 0.0, 0.0009255302192081467, 0.0009255302192081467, 0.000833143326004818, 0.0009170677861057888, 0.0009170677861057888, 0.000803895873637383, 0.0008038958736374154, 0.0008684484505757075])

In [84]:
### QTCI K3a: ###

dict_K3 = Dict("R"=>10, "extent"=>extent, "parquetdata"=>filename)

R_K3 = dict_K3["R"]
qR_K3 = 2^R_K3
qmesh = range(-extent, extent, length=qR_K3) |> collect
println("extent of qmesh: \t ±", extent)
println("resol of qmesh: \t", (2. *extent) / qR_K3)
println()
println("resol of ω/ν: \t", minimum([δω_K3, δν_K3]))




extent of qmesh: 	 ±10
resol of qmesh: 	0.01953125

resol of ω/ν: 	0.0012733669196278727


In [20]:


qttK3a, _, _ = quanticscrossinterpolate(
    ComplexF64,
    (x, y, z) -> K3ainterp(x, y, z),
    [qmesh, qmesh, qmesh],
    tolerance=tol_high
)

qttK3a2, _, _ = quanticscrossinterpolate(
    ComplexF64,
    (x, y, z) -> K3ainterp(x, y, z),
    [qmesh, qmesh, qmesh],
    tolerance=tol_low
)

(QuanticsTCI.QuanticsTensorCI2{ComplexF64}(TensorCrossInterpolation.TensorCI2{ComplexF64} with rank 635, QuanticsTCI.UnfoldingSchemes.interleaved), [2, 4, 8, 16, 32, 64, 128, 256, 501, 627, 642, 647, 655, 648, 635], [0.0, 0.0, 0.0007015206479788752, 0.0007015206479788752, 0.00025923351996977176, 0.0004252733971183641, 0.0005117753818683334, 0.0005636336169576582, 0.0005636336169576582, 0.000506488346079366, 0.0005023329092798772, 0.0005023329092798772, 0.00048136715046555176, 0.00042234278542912335, 0.0006397813340586065])

In [87]:
function save_plotdata(qtt, Kclass_str, qttdata2D, dict_K)
    file = h5open("plotdata_K23a_"*keldyshComponentsK2a*"_u="*string(u_here)*".h5", "cw")
    g_K = create_group(g, Kclass_str)
    g_K["qttdata2D"] = qttdata2D
    g_K["qplotmesh"] = qplotmesh
    g_K["pivoterrors"] = qtt.tt.pivoterrors
    g_K["linkdims"] = TCI.linkdims(qttK2a.tt)
    g_K["R"] = R_K3 = 10
    save("qtt_dict_"*Kclass_str*".jld", "qtt_dict", dict_K)
    close(file)
    return nothing
end

save_plotdata (generic function with 1 method)

In [ ]:
zoom = 2^4
qplotmesh = Int.(round.(range(1, qR_K2, length=300)))
box = (-extent, extent, -extent, extent) ./ zoom
qttK2adata = qttK2a.(qplotmesh, qplotmesh')

zoom = 2^4
Length = 128
qplotmesh = Int.(round.(range(1, qR_K3, length=Length)))
box = (-extent, extent, -extent, extent) #./ zoom
qttK3adata = qttK3a.(qplotmesh, qplotmesh',2^(R_K3-1))




qR_K3 = 2^R_K3
qmesh = range(-extent, extent, length=qR_K3) |> collect
    


In [27]:
file = h5open("plotdata_K23a_"*keldyshComponentsK2a*"_u="*string(u_here)*".pdf", "w")


🗂️ HDF5.File: (read-write) plotdata_K23a_1112_u=1.0.pdf

In [10]:
zoom = 2^4
qplotmesh = Int.(round.(range(1, qR_K2, length=300)))
box = (-extent, extent, -extent, extent) ./ zoom

qttK2adata = qttK2a.(qplotmesh, qplotmesh')
fig, axs = subplots(ncols=2, nrows=2, figsize=(300, 250)./72)

axs[1, 1].set_title(L"\mathrm{Re}(K_{2a})")
axs[1, 1].imshow(real(qttK2adata)'; extent=box, kwargs...) |> colorbar
axs[1, 2].set_title(L"\mathrm{Im}(K_{2a})")
axs[1, 2].imshow(imag(qttK2adata)'; extent=box, kwargs...) |> colorbar

axs[1, 1].set_ylabel(L"\nu")
for ax in axs[1, :]
    ax.set_xlabel(L"\omega")
end

axs[2, 1].semilogy(1:TCI.rank(qttK2a.tt), qttK2a.tt.pivoterrors, label=L"\epsilon=10^{-2}")
axs[2, 1].semilogy(1:TCI.rank(qttK2a2.tt), qttK2a2.tt.pivoterrors, label=L"\epsilon=10^{-3}")
axs[2, 1].set_xlabel(L"D_{\max}")
axs[2, 1].set_ylabel("abs. error")

axs[2, 2].semilogy(1:2R_K2-1, 2 .^ min.(1:2R_K2-1, 2R_K2-1:-1:1), color="gray", linewidth=0.5)
axs[2, 2].semilogy(1:2R_K2-1, TCI.linkdims(qttK2a.tt))
axs[2, 2].semilogy(1:2R_K2-1, TCI.linkdims(qttK2a2.tt))
axs[2, 2].set_xlabel(L"\ell")
axs[2, 2].set_ylabel(L"D_\ell")
axs[2, 2].set_ylim(1.5, 300)
axs[2, 2].set_xticks([1, 10, 19])

axs[2, 1].legend()
fig.suptitle("QTCI of K2a "*keldyshComponentsK2a*" component")
tight_layout()


fig.savefig("K2a_"*keldyshComponentsK2a*"_u="*string(u_here)*".pdf")


In [22]:
zoom = 2^4
Length = 128
qplotmesh = Int.(round.(range(1, qR_K3, length=Length)))
box = (-extent, extent, -extent, extent) #./ zoom

qttK3adata = qttK3a.(qplotmesh, qplotmesh',2^(R_K3-1))
fig, axs = subplots(ncols=2, nrows=2, figsize=(300, 250)./72)

axs[1, 1].set_title(L"\mathrm{Re}(K_{3a;\, \omega,\nu,\nu'})")
axs[1, 1].imshow(real(qttK3adata)'; extent=box, kwargs...) |> colorbar
axs[1, 2].set_title(L"\mathrm{Im}(K_{3a})")
axs[1, 2].imshow(imag(qttK3adata)'; extent=box, kwargs...) |> colorbar

axs[1, 1].set_ylabel(L"\nu'")
for ax in axs[1, :]
    ax.set_xlabel(L"\nu")
end
axs[2, 1].semilogy(1:TCI.rank(qttK3a.tt), qttK3a.tt.pivoterrors, label=L"\epsilon=10^{-2}")
axs[2, 1].semilogy(1:TCI.rank(qttK3a2.tt), qttK3a2.tt.pivoterrors, label=L"\epsilon=10^{-3}")
axs[2, 1].set_xlabel(L"D_{\max}")
axs[2, 1].set_ylabel("abs. error")

fig.savefig("K3a_"*keldyshComponentsK3a*"_u="*string(u_here)*".png")

axs[2, 2].semilogy(1:3R_K3-1, 2 .^ min.(1:3R_K3-1, 3R_K3-1:-1:1), color="gray", linewidth=0.5)
axs[2, 2].semilogy(1:3R_K3-1, TCI.linkdims(qttK3a.tt))
axs[2, 2].semilogy(1:3R_K3-1, TCI.linkdims(qttK3a2.tt))
axs[2, 2].set_xlabel(L"\ell")
axs[2, 2].set_ylabel(L"D_\ell")
#axs[2, 2].set_ylim(1.5, 300)
axs[2, 2].set_xticks([1, 10, 19])

axs[2, 1].legend()
fig.suptitle("QTCI of K3a "*keldyshComponentsK3a*" component")
tight_layout()


fig.savefig("K3a_"*keldyshComponentsK3a*"_u="*string(u_here)*".pdf")

In [91]:
save_plotdata(qttK2a,  "u"*string(u_here)*"_K2a" *"_"*keldyshComponentsK2a*"_3", qttK2adata, Dict("R"=>R_K2, "extent"=>extent, "parquetdata"=>filename, "tol"=>tol_high))
save_plotdata(qttK2a2, "u"*string(u_here)*"_K2a2"*"_"*keldyshComponentsK2a*"_3", qttK2adata, Dict("R"=>R_K2, "extent"=>extent, "parquetdata"=>filename, "tol"=>tol_low))
save_plotdata(qttK3a,  "u"*string(u_here)*"_K3a" *"_"*keldyshComponentsK3a*"_3", qttK3adata, Dict("R"=>R_K3, "extent"=>extent, "parquetdata"=>filename, "tol"=>tol_high))
save_plotdata(qttK3a2, "u"*string(u_here)*"_K3a2"*"_"*keldyshComponentsK3a*"_3", qttK3adata, Dict("R"=>R_K3, "extent"=>extent, "parquetdata"=>filename, "tol"=>tol_low))